Author: Lai Khee Jiunn
  
  

Source: 

    [1] http://www.bursamalaysia.com/market/listed-companies/list-of-companies/main-market/
    [2] https://klse.i3investor.com/
    [3] https://www.klsescreener.com/
  
  

Reference:

    [1] K. M. Ho, "How to make money from your stock investment even in a falling market", 3rd ed., Malaysia, Kanyin Publications Sdn. Bhd., 2017.

Note: The entire process in this jupyter notebook may take up to 2.5 hours depending on your internet speed

In [1]:
import numpy as np, pandas as pd, re
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import time
from tqdm import tqdm

### Get all listed company names and stock codes from Bursa Malaysia

In [2]:
driver = webdriver.Chrome()
driver.switch_to.window(driver.current_window_handle)
driver.maximize_window()
driver.get('http://www.bursamalaysia.com/market/listed-companies/list-of-companies/main-market/')
code, name = [],[]
x = driver.find_element_by_tag_name('table').find_element_by_tag_name('tbody').find_elements_by_tag_name('a')
for i in tqdm(x):
    y = i.get_attribute('href')
    z = y.split('=')
    if len(z) > 1:
        code.append(z[-1])
        name.append(i.get_attribute('innerText'))
driver.quit()

100%|██████████████████████████████████████████████████████████████| 1600/1600 [00:25<00:00, 62.42it/s]


In [3]:
stock_code = pd.DataFrame({'code':code,'name':name})
stock_code.loc[370,'code'] = '5235SS'
stock_code.drop(index=371,inplace=True)
stock_code.reset_index(drop=True)
print(stock_code.shape)
stock_code.head(1)

(800, 2)


,code,name
0,5250,7-ELEVEN MALAYSIA HOLDINGS BERHAD


In [4]:
stock_code.to_csv('stock_code_table.csv',index=False)

### Phase 1: Filter stocks by ROI >= 10%

In [5]:
# Get general stock information from klse.i3investor.com
start_time = time()
driver = webdriver.Chrome()
driver.switch_to.window(driver.current_window_handle)
driver.maximize_window()
error, result = [],[]
for i in tqdm(stock_code.code):
    try:
        driver.get('https://klse.i3investor.com/servlets/stk/fin/'+str(i)+'.jsp?type=last10fy')
        cat = driver.find_element_by_class_name('boarAndSector').get_attribute('innerText').split(' : ')[-1]
        x = driver.find_elements_by_tag_name('table')[13].find_elements_by_tag_name('tr')
        if len(x) != 119:
            x = driver.find_elements_by_tag_name('table')[14].find_elements_by_tag_name('tr')
        year_end = x[1].find_elements_by_tag_name('th')[3].get_attribute('innerText')
        temp = x[15].find_elements_by_tag_name('td')
        net_profit = temp[3].get_attribute('innerText')
        temp = x[42].find_elements_by_tag_name('td')
        nosh = temp[3].get_attribute('innerText')
        temp = x[84].find_elements_by_tag_name('td')
        price = temp[3].get_attribute('innerText')
        result.append([i, cat, year_end, net_profit, nosh, price])
    except:
        error.append(i)
driver.quit()
print('Time taken: ',time()-start_time)

100%|██████████████████████████████████████████████████████████████| 800/800 [1:12:57<00:00,  6.16s/it]


Time taken:  4382.734860658646


In [6]:
data = pd.DataFrame(result, columns=['code','category','year_end','net_profit','num_share','price'])
data = pd.merge(data,stock_code,'left')
# net_profit and num_share are in thousands
data.net_profit = data.net_profit.apply(lambda x: int(x.replace(',','')))
data.num_share = data.num_share.apply(lambda x: int(x.replace(',','')))
data.price = data.price.apply(lambda x: float(x))
data['ROI'] = np.around(data.net_profit / (data.num_share * data.price) * 100, 2)
print(data.shape)
data.to_csv('klse_investor_roi.csv',index=False)
data.head(1)

(476, 8)


,code,category,year_end,net_profit,num_share,price,name,ROI
0,5250,Consumer,31/12/17,50107,1110385,1.54,7-ELEVEN MALAYSIA HOLDINGS BERHAD,2.93


In [7]:
data[data.ROI >= 10].category.value_counts()

Industrial Products         38
Property                    20
Consumer                    15
Construction                 9
Plantations                  6
Finance                      4
Transportation&Logistics     4
Technology                   3
Energy                       2
Telco&Media                  1
REITS                        1
Name: category, dtype: int64

In [8]:
data.ROI.describe()

count      476.000000
mean        11.832101
std        486.012354
min      -2380.550000
25%         -1.932500
50%          4.825000
75%          9.327500
max      10138.630000
Name: ROI, dtype: float64

In [10]:
data1 = data.copy()
data1 = data1.loc[data1.ROI > 10]
# The commented code below helps you to filter out the sectors which you want
# wanted = ['Consumer','Energy','Finance','Health Care','Industrial Products','Plantations','Technology']
# data1 = data1[data1.category.isin(wanted)]
data1.sort_values(['category','ROI'],ascending=[True,False],inplace=True)
data1['code'] = data1['code'].astype(str)
data1.reset_index(drop=True)
print(data1.shape)
data1.head(1)

(102, 8)


,code,category,year_end,net_profit,num_share,price,name,ROI
237,9261,Construction,31/05/18,95122,658851,0.665,GADANG HOLDINGS BHD,21.71


In [11]:
# Example of Carlsberg Brewery Malaysia Berhad
data[data.code=='2836']

,code,category,year_end,net_profit,num_share,price,name,ROI
110,2836,Consumer,31/12/17,221165,305748,15.3,CARLSBERG BREWERY MALAYSIA BERHAD,4.73


In [12]:
# Get stock prices in 52 weeks range from KLSE Screener, which will be used in Phase 4
start_time = time()
driver = webdriver.Chrome()
driver.switch_to.window(driver.current_window_handle)
driver.maximize_window()
error2, result2 = [],[]
for i in tqdm(data1.code):
    try:
        driver.get('https://www.klsescreener.com/v2/stocks/view/'+i)
        week52 = driver.find_element_by_class_name('stock_details').find_elements_by_tag_name('tr')[6].find_element_by_class_name('number').get_attribute('innerText').split(' - ')
        result2.append([i, float(week52[0]), float(week52[-1])])
    except:
        error.append(i)
driver.quit()
print('Time taken: ',time()-start_time)
data2 = pd.DataFrame(result2, columns=['code','52w low','52w high'])
data2['code'] = data2['code'].astype(str)
print(data2.shape)
data2.to_csv('klse_screener_52week_price.csv',index=False)
data2.head(1)

100%|████████████████████████████████████████████████████████████████| 102/102 [20:39<00:00, 15.43s/it]


Time taken:  1246.0349161624908
(102, 3)


,code,52w low,52w high
0,9261,0.475,1.18


### Phase 2: Get past 10 years data

In [13]:
# Get the last 10 years of financial data from klse.i3investor.com
start_time = time()
driver = webdriver.Chrome()
driver.switch_to.window(driver.current_window_handle)
driver.maximize_window()
error3 = []
data3 = pd.DataFrame()
for i in tqdm(data2.code):
    try:
        driver.get('https://klse.i3investor.com/servlets/stk/fin/'+str(i)+'.jsp?type=last10fy')
        cat = driver.find_element_by_class_name('boarAndSector').get_attribute('innerText').split(' : ')[-1]
        x = driver.find_elements_by_tag_name('table')[13].find_elements_by_tag_name('tr')
        if len(x) != 119:
            x = driver.find_elements_by_tag_name('table')[14].find_elements_by_tag_name('tr')
        year_end = [i,cat,'Year End']+[x.get_attribute('innerText') for x in x[1].find_elements_by_tag_name('th')[3:-1]]
        net_profit = [i,cat,'NP']+[x.get_attribute('innerText') for x in x[15].find_elements_by_tag_name ('td')[3:-1]]
        div = [i,cat,'Div']+[x.get_attribute('innerText') for x in x[31].find_elements_by_tag_name ('td')[3:-1]]
        div_pay = [i,cat,'Div Payout']+[x.get_attribute('innerText') for x in x[34].find_elements_by_tag_name ('td')[3:-1]]
#         net_worth = [i,cat,'Net Worth']+[x.get_attribute('innerText') for x in x[39].find_elements_by_tag_name ('td')[3:-1]]
        nosh = [i,cat,'NOSH']+[x.get_attribute('innerText') for x in x[42].find_elements_by_tag_name ('td')[3:-1]]
        np_margin = [i,cat,'NP Margin']+[x.get_attribute('innerText') for x in x[47].find_elements_by_tag_name ('td')[3:-1]]
        roe = [i,cat,'ROE']+[x.get_attribute('innerText') for x in x[50].find_elements_by_tag_name ('td')[3:-1]]
        eps = [i,cat,'EPS']+[x.get_attribute('innerText') for x in x[58].find_elements_by_tag_name ('td')[3:-1]]
        eps_gr = [i,cat,'EPS GR']+[x.get_attribute('innerText') for x in x[59].find_elements_by_tag_name ('td')[3:-1]]
        dps = [i,cat,'DPS']+[x.get_attribute('innerText') for x in x[61].find_elements_by_tag_name ('td')[3:-1]]
        price = [i,cat,'Price']+[x.get_attribute('innerText') for x in x[84].find_elements_by_tag_name ('td')[3:-1]]
        pe = [i,cat,'P/E']+[x.get_attribute('innerText') for x in x[88].find_elements_by_tag_name ('td')[3:-1]]
        dy = [i,cat,'DY']+[x.get_attribute('innerText') for x in x[94].find_elements_by_tag_name ('td')[3:-1]]
        data3 = pd.concat([data3, pd.DataFrame([year_end, net_profit, div, div_pay, nosh, np_margin, roe, eps, eps_gr, dps, price, pe, dy])])
    except:
        error.append(i)
driver.quit()
print('Time taken: ',time()-start_time)
data3.columns = ['code','category','detail','Y1','Y2','Y3','Y4','Y5','Y6','Y7','Y8','Y9','Y10']
data3['code'] = data3['code'].astype(str)
print(data3.shape)
data3.to_csv('klse_investor_10yr.csv',index=False)
data3.head(1)

100%|████████████████████████████████████████████████████████████████| 102/102 [28:28<00:00,  5.88s/it]


Time taken:  1719.3734123706818
(1326, 13)


,code,category,detail,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10
0,9261,Construction,Year End,31/05/18,31/05/17,31/05/16,31/05/15,31/05/14,31/05/13,31/05/12,31/05/11,31/05/10,31/05/09


In [14]:
data3[data3.code=='2836']

,code,category,detail,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10


### Phase 3: Process each stock

In [16]:
error4, result4, symbol= [], [], ['  -  ', '- %']
for i in tqdm(data3.code.unique()):
    stock = data3[data3.code == i].fillna('  -  ')
    try:
        for y in [float(x.replace('%','').replace(',','')) for x in stock[stock.detail == 'NP'].values[0][3:].tolist() if x not in symbol]: assert(y >= 0)
        try:
            for y in [float(x.replace('%','').replace(',','')) for x in stock[stock.detail == 'EPS GR'].values[0][3:].tolist() if x not in symbol]: assert(y >= 0)  
            temp = [float(x.replace('%','')) for x in stock[stock.detail == 'ROE'].values[0][3:].tolist() if x not in symbol]
            nY = len(temp)
            try:
                assert(nY >= 5)
                roe_avg = np.around(np.average(temp), 2)
                temp = [float(x.replace('%','')) for x in stock[stock.detail == 'EPS GR'].values[0][3:3+nY].tolist() if x not in symbol]
                eps_gr_avg = np.around(sum(temp)/nY, 2)
                temp = [float(x.replace('%','')) for x in stock[stock.detail == 'NP Margin'].values[0][3:].tolist() if x not in symbol]
                np_margin_avg = np.around(np.average(temp), 2)
                temp = [float(x) for x in stock[stock.detail == 'P/E'].values[0][3:3+nY].tolist() if x not in symbol]
                pe_now = temp[0]
                pe_avg = np.around((max(temp) + min(temp))/2, 2)
                eps_now = float(stock[stock.detail == 'EPS'].values[0][3])
                dps_now = float(stock[stock.detail == 'DPS'].values[0][3])
                eps = [float(x) for x in stock[stock.detail == 'EPS'].values[0][3:3+nY].tolist() if x not in symbol]
                dps = [float(x) for x in stock[stock.detail == 'DPS'].values[0][3:3+nY].tolist() if x not in symbol]
                dpayout = np.around(np.average([dps[i]/eps[i]*100 for i in range(nY)]),2)
                temp = [float(x) for x in stock[stock.detail == 'DY'].values[0][3:3+nY].tolist() if x not in symbol]
                dy = np.around(np.average(temp),2)
                result4.append([i, nY, roe_avg, eps_gr_avg, np_margin_avg, pe_avg, pe_now, eps_now, dps_now, dpayout, dy])
            except:
                error4.append([i,'Less than 5 years'])
        except:
            error4.append([i,'Negavtive EPS Growth Rate'])
    except:
        error4.append([i,'Negative Net Profit'])

100%|███████████████████████████████████████████████████████████████| 102/102 [00:00<00:00, 232.97it/s]


In [17]:
er_df = pd.DataFrame(error4, columns=['code','reason'])
er_df.reason.value_counts()

Negative Net Profit          51
Negavtive EPS Growth Rate    51
Name: reason, dtype: int64

From the observation above, it is shown that the stocks filtered do not have consitent positve EPS growth rate, which is different from what is taught from the book. Thus, the eps growth rate filter is suspended.

In [18]:
error4, result4, symbol= [], [], ['  -  ', '- %']
for i in tqdm(data3.code.unique()):
    stock = data3[data3.code == i].fillna('  -  ')
    try:
        for y in [float(x.replace('%','').replace(',','')) for x in stock[stock.detail == 'NP'].values[0][3:].tolist() if x not in symbol]: assert(y >= 0)
#         try:
#             for y in [float(x.replace('%','').replace(',','')) for x in stock[stock.detail == 'EPS GR'].values[0][3:].tolist() if x not in symbol]: assert(y >= 0)  
        temp = [float(x.replace('%','')) for x in stock[stock.detail == 'ROE'].values[0][3:].tolist() if x not in symbol]
        nY = len(temp)
        try:
            assert(nY >= 5)
            roe_avg = np.around(np.average(temp), 2)
            temp = [float(x.replace('%','')) for x in stock[stock.detail == 'EPS GR'].values[0][3:3+nY].tolist() if x not in symbol]
            eps_gr_avg = np.around(sum(temp)/nY, 2)
            temp = [float(x.replace('%','')) for x in stock[stock.detail == 'NP Margin'].values[0][3:].tolist() if x not in symbol]
            np_margin_avg = np.around(np.average(temp), 2)
            temp = [float(x) for x in stock[stock.detail == 'P/E'].values[0][3:3+nY].tolist() if x not in symbol]
            pe_now = temp[0]
            pe_avg = np.around((max(temp) + min(temp))/2, 2)
            eps_now = float(stock[stock.detail == 'EPS'].values[0][3])
            dps_now = float(stock[stock.detail == 'DPS'].values[0][3])
            eps = [float(x) for x in stock[stock.detail == 'EPS'].values[0][3:3+nY].tolist() if x not in symbol]
            dps = [float(x) for x in stock[stock.detail == 'DPS'].values[0][3:3+nY].tolist() if x not in symbol]
            dpayout = np.around(np.average([dps[i]/eps[i]*100 for i in range(nY)]),2)
            temp = [float(x) for x in stock[stock.detail == 'DY'].values[0][3:3+nY].tolist() if x not in symbol]
            dy = np.around(np.average(temp),2)
            result4.append([i, nY, roe_avg, eps_gr_avg, np_margin_avg, pe_avg, pe_now, eps_now, dps_now, dpayout, dy])
        except:
            error4.append([i,'Less than 5 years'])
#         except:
#             error4.append([i,'Negavtive EPS Growth Rate'])
    except:
        error4.append([i,'Negative Net Profit'])

100%|███████████████████████████████████████████████████████████████| 102/102 [00:00<00:00, 123.22it/s]


In [19]:
er_df = pd.DataFrame(error4, columns=['code','reason'])
er_df.reason.value_counts()

Negative Net Profit    51
Less than 5 years      10
Name: reason, dtype: int64

In [20]:
data5 = pd.DataFrame(result4, columns=['code','num_year','ROE avg','EPS GR avg','NPM avg','P/E avg','P/E','EPS','DPS','DPayout avg','DY avg'])
data5['code'] = data5['code'].astype(str)
data5 = data5[data5['ROE avg'] >= 5]
data5 = data5[data5['EPS GR avg'] >= 5]
data5 = data5[data5['P/E'] < data5['P/E avg']]
data5 = data5.merge(data2,'left')
data5 = data5.merge(data1[['code','ROI','price','category','name']],'left')
print(data5.shape)
data5.sort_values('ROI',ascending=False,inplace=True)
data5.reset_index(drop=True,inplace=True)
data5.to_csv('klse_investor_stock_phase3.csv',index=False)
data5.head(1)

(24, 17)


,code,num_year,ROE avg,EPS GR avg,NPM avg,P/E avg,P/E,EPS,DPS,DPayout avg,DY avg,52w low,52w high,ROI,price,category,name
0,4715,10,16.86,18.07,18.85,11.3,2.18,20.48,17.0,40.75,2.45,2.76,5.6,45.84,5.63,Consumer,GENTING MALAYSIA BERHAD


In [22]:
# This means that Carlsberg stock did not pass our filter
data5[data5.code=='2836']

,code,num_year,ROE avg,EPS GR avg,NPM avg,P/E avg,P/E,EPS,DPS,DPayout avg,DY avg,52w low,52w high,ROI,price,category,name


### Phase 4: Intrinsic Value & Fair Value

In [23]:
result5 = []
for n, x in tqdm(data5.iterrows()):
    price_10y = [np.around(x['price'] * ((1 + (x['EPS GR avg'] / 100))**i) ,2) for i in range(5)]
    total_eps = np.around(sum(price_10y), 2)
    expected_price = np.around(price_10y[-1] * x['P/E avg'],2)
    total_dvd = np.around(total_eps * (x['DPayout avg'] / 100), 2)
    total_return = np.around(total_eps + total_dvd, 2)
    intrinsic = np.around(total_return / ((1 + (x['EPS GR avg'] / 100))**5), 2)
    price_A = np.around(intrinsic * 0.75, 2)
    price_B = np.around((x['52w high'] - x['52w low']) * 0.33 + x['52w low'], 2)
    result5.append([str(x['code']), intrinsic, price_A, price_B])

24it [00:00, 1718.60it/s]


In [24]:
data6 = data5.merge(pd.DataFrame(result5, columns=['code','intrinsic','price_A','price_B']))
print(data6.shape)
data6.to_csv('klse_investor_intrinsic.csv',index=False)
data6.head(1)

(24, 20)


,code,num_year,ROE avg,EPS GR avg,NPM avg,P/E avg,P/E,EPS,DPS,DPayout avg,DY avg,52w low,52w high,ROI,price,category,name,intrinsic,price_A,price_B
0,4715,10,16.86,18.07,18.85,11.3,2.18,20.48,17.0,40.75,2.45,2.76,5.6,45.84,5.63,Consumer,GENTING MALAYSIA BERHAD,24.75,18.56,3.7


In [25]:
# Empty because it did not pass the previous filter, you can try with the other 24 stocks
data6[data6.code=='2836']

,code,num_year,ROE avg,EPS GR avg,NPM avg,P/E avg,P/E,EPS,DPS,DPayout avg,DY avg,52w low,52w high,ROI,price,category,name,intrinsic,price_A,price_B


In [26]:
# Stocks which have prices lower than fair value price B
data6[data6.price < data6.price_B]

,code,num_year,ROE avg,EPS GR avg,NPM avg,P/E avg,P/E,EPS,DPS,DPayout avg,DY avg,52w low,52w high,ROI,price,category,name,intrinsic,price_A,price_B
19,5007,10,9.01,64.12,8.14,60.22,8.18,18.83,8.0,30.56,3.04,1.49,2.00,12.23,1.540,Industrial Products,CHIN WELL HOLDINGS BERHAD,2.87,2.15,1.66
20,7579,10,10.73,38.68,8.12,9.79,8.40,7.99,0.5,21.01,1.94,0.60,0.93,11.90,0.665,Industrial Products,AWC BERHAD,1.67,1.25,0.71


In [27]:
# Stocks which have prices lower than fair value price B and average NPM of 10% or higher
data6[(data6.price < data6.price_B) & (data6['NPM avg'] >= 10)]

,code,num_year,ROE avg,EPS GR avg,NPM avg,P/E avg,P/E,EPS,DPS,DPayout avg,DY avg,52w low,52w high,ROI,price,category,name,intrinsic,price_A,price_B


In practical, intrinsic value and price A are way beyond from the current price because there is no consistency of positive growth rate for any stock, my suggestion will be excluding the idea of positive growth rate, but filter stocks with positive earnings or net profit margin at 5% or 10%  
 <br />
Another method would be filtering stocks according to the dividend yield over the past 10 years. Although this method is not recommended by the author,but it returns more stock options and most of them has dividend yield of 10% or higher

### Process by Dividend Yield

In [28]:
data7 = pd.DataFrame(result4, columns=['code','num_year','ROE avg','EPS GR avg','NPM avg','P/E avg','P/E','EPS','DPS','DPayout avg','DY avg'])
data7['code'] = data7['code'].astype(str)
# The commented code below are some filters which can be tuned
# data7 = data7[data7['ROE avg'] >= 5]
# data7 = data7[data7['EPS GR avg'] >= 5]
# data7 = data7[data7['P/E'] < data5['P/E avg']]
data7 = data7.merge(data2,'left')
data7 = data7.merge(data1[['code','ROI','price','category','name']],'left')
print(data7.shape)
data7.sort_values('ROI',ascending=False,inplace=True)
data7.reset_index(drop=True,inplace=True)
data7.head(1)

(41, 17)


,code,num_year,ROE avg,EPS GR avg,NPM avg,P/E avg,P/E,EPS,DPS,DPayout avg,DY avg,52w low,52w high,ROI,price,category,name
0,4715,10,16.86,18.07,18.85,11.3,2.18,20.48,17.0,40.75,2.45,2.76,5.6,45.84,5.63,Consumer,GENTING MALAYSIA BERHAD


In [30]:
data7.sort_values('DY avg',ascending=False)

,code,num_year,ROE avg,EPS GR avg,NPM avg,P/E avg,P/E,EPS,DPS,DPayout avg,DY avg,52w low,52w high,ROI,price,category,name
14,5127,10,9.50,28.36,98.31,14.04,7.02,12.95,5.50,77.81,7.67,0.810,0.910,14.24,0.910,REITS,AMANAHRAYA REAL ESTATE INVESTMENT TRUST
4,5254,5,9.11,61.73,35.95,20.30,3.97,41.58,19.50,154.67,6.68,0.655,1.780,25.20,1.650,Plantations,BOUSTEAD PLANTATIONS BERHAD
33,7089,10,16.70,29.61,7.22,6.31,9.05,39.79,20.00,35.22,6.49,2.250,3.310,11.05,3.600,Consumer,LII HEN INDUSTRIES BHD.
7,8591,10,6.72,22.11,7.10,9.62,5.54,16.30,4.00,33.14,5.53,0.790,1.100,18.04,0.905,Construction,CREST BUILDER HOLDINGS BERHAD
22,7169,10,9.82,13.35,4.00,9.74,7.94,16.87,7.00,37.27,5.22,1.210,1.390,12.60,1.310,Industrial Products,DOMINANT ENTERPRISE BERHAD
23,7087,10,18.55,21.41,6.56,6.49,8.03,56.15,20.00,35.57,5.06,3.880,5.520,12.45,4.510,Consumer,MAGNI-TECH INDUSTRIES BERHAD
31,7129,10,15.25,-5.64,18.05,10.88,8.79,30.37,15.00,52.11,4.85,2.380,2.980,11.37,2.670,Consumer,ASIA FILE CORPORATION BHD
39,7210,10,13.18,0.33,5.73,8.76,9.92,10.58,5.00,38.42,4.79,0.505,1.280,10.08,1.050,Transportation&Logistics,FREIGHT MANAGEMENT HOLDINGS BERHAD
36,7229,10,16.07,10.83,9.04,7.27,9.58,28.50,13.50,30.86,4.72,2.200,2.740,10.44,2.730,Industrial Products,FAVELLE FAVCO BERHAD
19,6939,10,10.06,8.67,11.67,8.34,7.74,6.46,2.25,28.83,4.71,0.430,0.570,12.92,0.500,Consumer,FIAMMA HOLDINGS BERHAD


 #### Thank you for reading, using, and exploring this, hope to see you soon